# Datasets

In [241]:
import torch
import numpy as np
from torch.utils.data import Dataset
from dataclasses import dataclass
from torchvision import datasets, transforms
from PIL import Image
from torchvision.transforms import ToPILImage
from DynGenModels.trainer.configs import Training_Configs

@dataclass
class Configs(Training_Configs):
    DATA_SOURCE = None
    DATA_TARGET = 'mnist'
    dim_input = (1, 28, 28)
    batch_size = 10
    MODEL : str = 'Unet'
    dim_hidden : int = 32
    num_res_blocks : int = 1
    DYNAMICS : str = 'CondFlowMatch'
    sigma : float = 0.001
    augmented : bool = False
    t0 : float = 0.0
    t1 : float = 1.0

class MNIST_Dataset(Dataset):

    def __init__(self, configs: dataclass):
        self.configs = configs
        self.get_target_data()
        self.get_source_data()

    def __getitem__(self, idx):
        output = {}
        output['target'] = self.target[idx]
        output['source'] = self.source[idx]
        output['context'] = self.target_label[idx]
        output['mask'] = torch.ones_like(output['target'])
        return output

    def __len__(self):
        return len(self.target)
    
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

    def get_target_data(self):
        
        if self.configs.DATA_TARGET == 'mnist':
            self.data_1 = datasets.MNIST(root='../data', train=True, download=True, transform=transforms.ToTensor())
        
        elif self.configs.DATA_TARGET == 'emnist':
            self.data_1 = datasets.EMNIST(root='../data', split='letters', train=True, download=True, transform=transforms.ToTensor())
        
        elif self.configs.DATA_TARGET == 'fashion':
             self.data_1 = datasets.FashionMNIST(root='../data', train=True, download=True, transform=transforms.ToTensor())
        
        self.target = [d[0] for d in self.data_1]
        self.target_label = [d[1] for d in self.data_1]
        
    def get_source_data(self):

        if self.configs.DATA_SOURCE == 'mnist':
            self.data_0 = datasets.MNIST(root='../data', train=True, download=True, transform=transforms.ToTensor())
       
        elif self.configs.DATA_SOURCE == 'emnist':
            self.data_0 = datasets.EMNIST(root='./data', split='letters', train=True, download=True, transform=transforms.ToTensor())
       
        elif self.configs.DATA_SOURCE == 'fashion':
             self.data_0 = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
        else:
            self.data_0 = self.data_1

        self.source = [d[0] for d in self.data_0] if self.configs.DATA_SOURCE is not None else [torch.rand_like(d[0]) for d in self.data_0]
        self.source_label = [d[1] for d in self.data_0] if self.configs.DATA_SOURCE is not None else [0] * len(self.data_0)

In [242]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Subset
from torch.utils.data import Dataset
from dataclasses import dataclass

class MNISTDataLoader:

    def __init__(self, datasets: Dataset, configs: dataclass): # type: ignore

        self.datasets = datasets        
        self.fracs = configs.data_split_fracs
        self.batch_size = configs.batch_size
        self.dataloader()

    def train_val_test_split(self, shuffle=False):
        assert np.abs(1.0 - sum(self.fracs)) < 1e-3, "Split fractions do not sum to 1!"
        total_size = len(self.datasets)
        train_size = int(total_size * self.fracs[0])
        valid_size = int(total_size * self.fracs[1])

        #...define splitting indices

        idx = torch.randperm(total_size) if shuffle else torch.arange(total_size)
        idx_train = idx[:train_size].tolist()
        idx_valid = idx[train_size : train_size + valid_size].tolist()
        idx_test = idx[train_size + valid_size :].tolist()
        
        #...Create Subset for each split

        train_set = Subset(self.datasets, idx_train)
        valid_set = Subset(self.datasets, idx_valid) if valid_size > 0 else None
        test_set = Subset(self.datasets, idx_test) if self.fracs[2] > 0 else None

        return train_set, valid_set, test_set


    def dataloader(self):

        print("INFO: building dataloaders...")
        print("INFO: train/val/test split ratios: {}/{}/{}".format(self.fracs[0], self.fracs[1], self.fracs[2]))
        
        train, valid, test = self.train_val_test_split(shuffle=True)
        self.train = DataLoader(dataset=train, batch_size=self.batch_size, shuffle=True)
        self.valid = DataLoader(dataset=valid,  batch_size=self.batch_size, shuffle=False) if valid is not None else None
        self.test = DataLoader(dataset=test,  batch_size=self.batch_size, shuffle=True) if test is not None else None

        print('INFO: train size: {}, validation size: {}, testing sizes: {}'.format(len(self.train.dataset),  # type: ignore
                                                                                    len(self.valid.dataset if valid is not None else []),  # type: ignore
                                                                                    len(self.test.dataset if test is not None else []))) # type: ignore


In [243]:
config = Configs()
config.workdir = './'


In [244]:
config.DATA_SOURCE = None
config.DATA_TARGET = 'mnist'
data = MNIST_Dataset(config)
dataloader = MNISTDataLoader(data, config)

transform = ToPILImage()

for batch in dataloader.train:
    pair = torch.cat([batch['source'][0].squeeze(), batch['target'][0].squeeze()], dim=1)
    img = transform(pair)
    break
img

INFO: building dataloaders...
INFO: train/val/test split ratios: 1.0/0.0/0.0
INFO: train size: 60000, validation size: 0, testing sizes: 0


In [237]:
config = Configs()
config.DATA_SOURCE = 'fashion'
config.DATA_TARGET = 'mnist'
data = MNIST_Dataset(config)
dataloader = MNISTDataLoader(data, config)

transform = ToPILImage()

for batch in dataloader.train:
    pair = torch.cat([batch['source'][0].squeeze(), batch['target'][0].squeeze()], dim=1)
    img = transform(pair)
    break
img

INFO: building dataloaders...
INFO: train/val/test split ratios: 1.0/0.0/0.0
INFO: train size: 60000, validation size: 0, testing sizes: 0


# U-net model

In [245]:
from torch import nn
from torch.nn import functional as F
from torchcfm.models.unet import UNetModel

class Unet(nn.Module):

    def __init__(self, configs):
        super().__init__()
        self.device = configs.DEVICE
        self.unet = UNetModel(dim=configs.dim_input, num_channels=configs.dim_hidden, num_res_blocks=configs.num_res_blocks)
        self.to(self.device)

    def forward(self, t, x, context=None, mask=None):
        x = self.unet(t, x, y=context)
        return x


In [247]:
Unet(config)

ValueError: num_channels must be divisible by num_groups

In [240]:
from DynGenModels.trainer.trainer import DynGenModelTrainer
from DynGenModels.dynamics.cnf.condflowmatch import OptimalTransportFlowMatching as dynamics

cfm = DynGenModelTrainer(dynamics = dynamics(config),
                         model = Unet(config), 
                         dataloader = dataloader, 
                         configs = config)

cfm.train()

ValueError: num_channels must be divisible by num_groups